<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/nbs/02b_misc_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#default_exp misc_utils

# Miscellaneneous Utilities

## Install fastai

Use latest fastai and fastcore versions

In [ ]:
#colab
# !pip install -Uqq git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastai --upgrade

     |████████████████████████████████| 194kB 5.8MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 


In [ ]:
#hide
#colab
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
#hide
!pip freeze | grep torch
!pip freeze | grep fast

torch==1.7.0+cu101
torch-xla==1.7
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101
fastai==2.2.5
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


## Patching BaseOptimizer to be Pickable
Patching Base Optimizer `__getstate__` and `__setstate__` whichi is used in pickling
the optimizer which should fix the bug in running the learner in multiple TPU cores
in XLA by which the  `def _fetch_gradients(optimizer)` in `for param_group in optimizer.__getstate__()['param_groups']:` fails, and this patch fixes the "copy constructor" to include the param_groups.

In [ ]:
#export
from fastai.optimizer import _BaseOptimizer
from fastcore.basics import patch
@patch
def __getstate__(self:_BaseOptimizer):
    d = {
            'state': self.state_dict(),
            'param_groups': self.param_groups,
        }
    if hasattr(self,'defaults'):
        d['defaults'] = self.defaults
    return d

@patch
def __setstate__(self:BaseOptimizer, data):
    if 'defaults' in data:
        self.defaults = data['defaults']
    self.load_state_dict(data['state'])
    self.param_groups = data['param_groups']

In [ ]:
#TODO add tests for pickling optimizers